In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

In [3]:
data = sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")

In [81]:
#data_m = data.limit(5000).drop("artistId")
data_b = data.withColumnRenamed("trackId","b_trackId")

In [42]:
from pyspark.sql.functions import desc

In [82]:
train_data=data.alias("a").join(data_b.alias("b"),"userId")\
.where("a.timestamp!=b.timestamp and trackId!=b_trackId")\
.where("b.timestamp - a.timestamp < 420 and b.timestamp - a.timestamp >= 0 ")

## Normalization could be done by next function

In [83]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number, sum

def norm(df, key1, key2, field, n): 
    
    window = Window.partitionBy(key1).orderBy(col(field).desc())
        
    topsDF = df.withColumn("row_number", row_number().over(window)) \
        .filter(col("row_number") <= n) \
        .drop(col("row_number")) 
        
    tmpDF = topsDF.groupBy(col(key1)).agg(col(key1), sum(col(field)).alias("sum_" + field))
   
    normalizedDF = topsDF.join(tmpDF, key1, "inner") \
        .withColumn("norm_" + field, col(field) / col("sum_" + field)) \
        .cache()

    return normalizedDF

In [96]:
from pyspark.sql import Window
from pyspark.sql.functions import col, rank

window = Window.partitionBy(col("trackId")).orderBy(col("count"))

track = train_data.groupBy(col("trackId"),col("b_trackId")).count()\
        .withColumn("position", rank().over(window))\
        .filter(col("position") < 50)

trackNorm = norm(track, "trackId", "b_trackId", "count", 1000) \
        .withColumn("id", col("trackId")) \
        .withColumn("id2", col("b_trackId")) \
        .withColumn("norm_count", col("norm_count") * 0.5) \
        .select(col("id"), col("id2"), col("norm_count"))     

window = Window.orderBy(col("norm_count").desc())
    
trackList = trackNorm.withColumn("position2", rank().over(window))\
    .orderBy(desc("norm_count"),col("id"), col("id2"))\
    .select(col("id"), col("id2"))\
    .take(40)

In [97]:
for val in trackList:
    print "%s %s" % val

798256 923706
798258 808254
798302 836228
798322 876562
798331 827364
798335 840741
798376 888871
798379 812055
798398 926302
798403 868805
798405 867217
798426 910880
798447 832635
798457 918918
798471 801831
798474 963162
798475 827475
798505 905671
798508 810743
798516 860347
798526 937573
798542 946408
798544 841232
798550 936295
798552 830267
798618 930224
798667 874844
798682 934393
798704 937570
798707 839389
798720 958333
798725 933147
798731 853117
798782 956938
798801 950802
798820 890393
798821 883244
798827 908022
798851 801321
798978 854212
